In [1]:
# Load the model


In [2]:
# Load the image
def get_img():
    import requests

    url = "https://img03.platesmania.com/221209/m/20496102.jpg"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36",
        "Referer": "https://www.google.com"
    }
    
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        with open("img.jpg", "wb") as f:
            f.write(response.content)
        print("Image downloaded successfully!")
    else:
        print("Failed to download image:", response.status_code)

In [3]:
import os
import cv2 
from ultralytics import YOLO

In [5]:
def crop_image_with_mode(model_path, image_path, output_dir='cropped_outputs'):
    # Create the output directory if it does not exist
    os.makedirs(output_dir, exist_ok=True)
    
    # model_path = 'COEN490/Koala/ALPR/YOLOv8n_TEST/my_project/exp1/weights/best.pt'
    model = YOLO(model_path)

    # Read the image using cv2
    # (Note: cv2 reads images in BGR format by default)
    image = cv2.imread(image_path)
    
    # Run inference (prediction) on the image
    # The results returned is a list of results (one per image in the batch)
    results = model.predict(source=image_path, conf=0.25, iou=0.45)
    
    for result in results:
        # The 'boxes' attribute contains the bounding boxes.
        # The .xyxy attribute returns the bounding boxes in [x1, y1, x2, y2] format.
        boxes = result.boxes.xyxy.cpu().numpy()  # Convert to NumPy array if on GPU
    
        # Loop through each detected bounding box
        for idx, box in enumerate(boxes):
            # Extract coordinates and cast them to integer
            x1, y1, x2, y2 = map(int, box)
            
            # Crop the image using the bounding box coordinates
            # Note: image is a NumPy array with shape (height, width, channels)
            cropped_image = image[y1:y2, x1:x2]
            # Define a filename for the cropped image
            output_path = os.path.join(output_dir, f'cropped_object_{idx+1}.jpg')
            
            # Save the cropped image using cv2.imwrite
            cv2.imwrite(output_path, cropped_image)
            print(f'Saved cropped image to {output_path}')
            

In [4]:
mod_path = "/speed-scratch/z_amm/COEN490/Koala/ALPR/YOLOv8n_TEST/my_project/exp1/weights/best.pt"
image_path = '/speed-scratch/z_amm/COEN490/Koala/ALPR/YOLOv8n_TEST/datasets/other/img.jpg'
crop_image_with_mode(mod_path, image_path)

NameError: name 'crop_image_with_mode' is not defined

In [101]:
# find contours

def upscale_img(im):
    scale_factor = 5.0
    new_width = int(im.shape[1] * scale_factor)
    new_height = int(im.shape[0] * scale_factor)

    # Resize the image using cubic interpolation (better quality)
    upscaled_im = cv2.resize(im, (new_width, new_height), interpolation=cv2.INTER_CUBIC)

    return upscaled_im

def find_rect(cnt_points):
    """
    Returns a list of rectangles set as 4 points 
    """
    # first find the leftmost point out of all the contours
    
    print(cnt_points)
    biggest_rect = []
    for cnt in cnt_points: 
        leftmost = min(cnt, key=lambda p: p[0][0]*p[0][0] + p[0][1]*p[0][1])
        biggest_rect.append(sorted(cnt, key=lambda p: (leftmost[0][0] - p[0][0]) * (leftmost[0][0] - p[0][0]) + (leftmost[0][1] - p[0][1]) * (leftmost[0][1] - p[0][1]), reverse=True)[0:3])
    print("Printing biggest_rect")
    print(biggest_rect)
    return biggest_rect
    

def find_contours(image_path):
    """
        Finds the contours(edges) of cropped license plate
    """
    directory = "/nfs/speed-scratch/z_amm/COEN490/Koala/ALPR/YOLOv8n_TEST"
    os.chdir(directory)

    im = cv2.imread(image_path)
    im = upscale_img(im)
    assert im is not None, "file could not be read, check with os.path.exists()"
    imgray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(imgray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    blur = cv2.GaussianBlur(thresh,(5,5),0)
    th3 = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Filtering contours
    license_plate_contour = []
    for cnt in contours:
        # Get bounding rectangle
        x, y, w, h = cv2.boundingRect(cnt)
        
        # Define aspect ratio and size constraints
        aspect_ratio = w / float(h)
        if 1 < aspect_ratio < 6 and 13000 < cv2.contourArea(cnt) < 30000:  
            approx = cv2.approxPolyDP(cnt, 0.02 * cv2.arcLength(cnt, True), True)
            
            # Check if it's a quadrilateral (4 corners)
            if 3 < len(approx) < 7:
                license_plate_contour.append(approx)
                # break  # Assuming only one plate, stop searching

    for cnt in contours:
        print(f"{cv2.contourArea(cnt)}")
    # Draw the detected license plate
    # if license_plate_contour is not None:
    rectangles_extracted = find_rect(license_plate_contour)
    for cnt in rectangles_extracted: 
        cv2.drawContours(im, [cnt], -1, (0, 255, 0), 3)
        find_diagonal(cnt)

    print(license_plate_contour[1])
    
    print("Before saving image:")  
    print(os.listdir(directory))  

    cv2.imwrite("img_contour_out.jpg", im)


In [102]:
image_path = '/speed-scratch/z_amm/COEN490/Koala/ALPR/YOLOv8n_TEST/cropped_outputs/cropped_object_1.jpg'
find_contours(image_path)

24932.0
66.5
89.5
14760.5
18.5
2.5
21.0
75.0
69.0
2.0
49.5
14.5
111.5
68.0
273.5
259.0
49.5
24.0
2.5
108.5
31.5
13.5
185.5
[array([[[  0,   0]],

       [[ 45, 154]],

       [[207, 114]],

       [[190,  14]],

       [[206,   0]]], dtype=int32), array([[[182,   9]],

       [[196, 104]],

       [[ 46, 135]],

       [[ 33,  36]],

       [[ 87,  44]]], dtype=int32)]
Printing biggest_rect
[[array([[207, 114]], dtype=int32), array([[206,   0]], dtype=int32), array([[190,  14]], dtype=int32)], [array([[196, 104]], dtype=int32), array([[182,   9]], dtype=int32), array([[ 46, 135]], dtype=int32)]]


error: OpenCV(4.11.0) :-1: error: (-5:Bad argument) in function 'drawContours'
> Overload resolution failed:
>  - Can't parse 'contours'. Sequence item with index 0 has a wrong type
>  - Can't parse 'contours'. Sequence item with index 0 has a wrong type
